In [105]:
import requests
import time
from google.cloud import pubsub
from typing import List
import urllib.parse

## Test collection

In [5]:
headers = {'User-Agent': 'Mozilla/5.0'}

url = 'https://www.comichron.com/monthlycomicssales/2018/2018-06.html'

response = requests.get(url, headers=headers)

response.raise_for_status()

## Manual collection

In [30]:
year = 2006
month = 4

!gcloud pubsub topics publish vaqmr --message '{"collector":"web_scrape", "work_list":[{"url":"https://www.comichron.com/monthlycomicssales/{year}/1998-01.html","storage_key":"monthlycomicssales_1998-01"}]}'

messageIds:
- '882117827305425'


In [78]:
# urls = ['https://www.comichron.com/monthlycomicssales/2018/2018-06.html',
#         'https://www.comichron.com/monthlycomicssales/1998/1998-12.html']

publisher = pubsub.PublisherClient()
topic_path = publisher.topic_path('dotufp', 'vaqmr')
                                  
# for year in range(1998, 2019): 
for year in (2019,):
    for month in range(1, 13):
        time.sleep(60)
              
        data = f'{{"collector":"web_scrape","work_list":[{{"url":"https://www.comichron.com/monthlycomicssales/{year}/{year}-{month:02d}.html","storage_key":"monthlycomicssales/{year}-{month:02d}"}}]}}'
        
        print(data)
        publisher.publish(topic_path, data=data.encode('utf-8'))


/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/2019/2019-01.html","storage_key":"monthlycomicssales/2019-01"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/2019/2019-02.html","storage_key":"monthlycomicssales/2019-02"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/2019/2019-03.html","storage_key":"monthlycomicssales/2019-03"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/2019/2019-04.html","storage_key":"monthlycomicssales/2019-04"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/2019/2019-05.html","storage_key":"monthlycomicssales/2019-05"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/2019/2019-06.html","storage_key":"monthlycomicssales/2019-06"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www

## Processing data

In [40]:
import random
import pandas as pd
from google.cloud import storage

In [56]:
storage_client = storage.Client(project='')

/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [51]:
def raw_monthly_sales(path_filter: str = 'web_scrape/monthlycomicssales/') -> list:
    storage_client = storage.Client()
    bucket = storage_client.bucket('dotufp-raw')
    raw_monthly_sales_list = list(bucket.list_blobs(prefix=path_filter))
    
    return raw_monthly_sales_list


In [53]:
def random_raw_monthly_sales(path_filter: str = 'web_scrape/monthlycomicssales/') -> str:
    raw_monthly_sales_list = raw_monthly_sales(path_filter)
    random_choice = random.choice(raw_monthly_sales_list)
    
    print(random_choice)
    return random_choice.download_as_string()


In [110]:
def all_raw_monthly_sales(path_filter: str = 'web_scrape/monthlycomicssales/') -> List[pd.DataFrame]:
    raw_monthly_sales_list = raw_monthly_sales(path_filter)
    
    return [{'data':pd.read_html(month.download_as_string()), 'month': urllib.parse.unquote(month.path).split('/')[6]} for month in raw_monthly_sales_list]
    

### Exploration

In [57]:
text = random_raw_monthly_sales()

len(text), text[:1000]

/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/au

<Blob: dotufp-raw, web_scrape/monthlycomicssales/2000-07/2019-12-08T04:07:36Z.raw, 1575778057299079>


(144757,
 b'<!doctype html>\n<html lang="en"><!-- InstanceBegin template="/Templates/index.dwt" codeOutsideHTMLIsLocked="false" -->\n<head>\n<meta charset="UTF-8">\n<!-- InstanceBeginEditable name="doctitle" -->\n<title>Comichron: July 2000 Comic Book Sales to Comics Shops</title>\n<meta name="description" content="Estimated comic book and graphic novel sales to North American comic shops">\n<link rel="canonical" href="https://www.comichron.com/monthlycomicssales/2000/2000-07.html" />\n<!-- InstanceEndEditable -->\n<meta name="keywords" content="comics sales, comic book sales, comic book circulation, comics rankings, comics industry, diamond comics charts, comics charts, comics analysis, comics history">\n<link href="/newstyles3.css" rel="stylesheet" type="text/css">\n<link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/v/dt/jq-2.2.4/dt-1.10.13/b-colvis-1.2.4/datatables.min.css"/>\n\n<script type="text/javascript" async src="https://cdn.datatables.net/v/dt/jq-2.2.4/d

In [58]:
data = pd.read_html(text)

In [59]:
len(data)

6

In [60]:
data[0]

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,NaN
1,#1,#2,#3,#4,#5


In [67]:
data[1].columns

Index(['Units', 'Dollars', 'Comic-book Title', 'Issue', 'Price', 'Publisher',
       'Est. units'],
      dtype='object')

In [61]:
data[1]

,Units,Dollars,Comic-book Title,Issue,Price,Publisher,Est. units
0,1,1,Spawn,100,$4.95,Image,143493
1,2,2,Uncanny X-Men,384,$2.25,Marvel,116652
2,3,3,X-Men,104,$2.25,Marvel,113872
3,4,10,Wolverine,154,$2.25,Marvel,82000
4,5,11,JLA,45,$2.25,DC,75397
...,...,...,...,...,...,...,...
295,296,323,Sidekicks,2,$2.75,Fantagraphics,2164
296,297,318,Boffy The Vampire Layer,2,$2.95,Fantagraphics,2120
297,298,319,Aquarium,4,$2.95,CPM,2115
298,299,268,Weasel,3,$4.95,Fantagraphics,2086


In [62]:
data[2]

,#1,#2,#3,#4


In [63]:
data[3]

,Units,Dollars,Trade Paperback title,Price,Publisher,Est. units
0,1,2,Sandman The Dream Hunters SC,$19.95,DC,5754
1,2,7,Buffy The Vampire Slayer Ring of Fire,$9.95,Dark Horse,5521
2,3,13,Gon On Safari,$7.95,DC,5186
3,4,3,Avengers The Kree Skrull War,$24.95,Marvel,4390
4,5,10,Star Wars Union,$12.95,Dark Horse,3784
5,6,15,Tomb Raider Vol. I The Saga of the Medusa Mask,$9.95,Image,3784
6,7,5,Grendel Black White and Red,$18.95,Dark Horse,3459
7,8,1,Hawkman Archives Vol. 1 HC,$49.95,DC,3309
8,9,11,Strangers In Paradise Vol. 8 My Other Life,$14.95,Abstract,3124
9,10,9,X-Men Visionaries Joe Madureira,$17.95,Marvel,2945


In [64]:
data[4]

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN


In [65]:
data[5]

Empty DataFrame
Columns: [(Share of Overall Final Order Dollars, DC, Marvel, Image, Dark Horse, Viz, Wizard, Chaos, Crossgen, Tokyopop, Dynamic Forces, Others, Share of Overall Preordered Units, Marvel, DC, Image, Dark Horse, Wizard, Crossgen, Viz, Chaos, Dynamic Forces, Other, Share of Overall Preordered Dollars, DC, Marvel, Image, Dark Horse, Wizard, Viz, Crossgen, Chaos, Dynamic Forces, Other, Items in Top 300, DC, Marvel, Image, Dark Horse, Archie, Viz, CPM, Fantagraphics, Crossgen, Antarctic, Sirius, Slave Labor, Chaos, Tokyopop, Radio, Awesome, Oni, Avatar, Gemstone, Harris, Ironcat, Realm, Kenzer, AAA Pop, Aardvark-Vanaheim, Acclaim, Cartoon, Crusade, Comic Cavalcade, Hamster, Insight, Horse, Humanoids, Jack Kirby, Wizard, TV, Dynamic Forces, Broken Halos, Share of Units in Top 300, Marvel, DC, Image, Dark Horse, Crossgen, Viz, Archie, Chaos, Tokyopop, Awesome, Harris, Sirius, Fantagraphics, CPM, Wizard, Oni, Slave Labor, Avatar, Kenzer, Broken Halos, Insight, Antarctic, AAA Pop, TV, Aardvark-Vanaheim, Radio, ...), (Share of Overall Final Order Dollars, 32.80%, 25.75%, 11.29%, 6.22%, 2.28%, 2.19%, 1.11%, 1.09%, 1.04%, 1.04%, 15.17%, Share of Overall Preordered Units, 32.12%, 32.01%, 14.22%, 5.16%, 2.10%, 1.38%, 1.37%, 0.71%, 0.25%, 10.68%, Share of Overall Preordered Dollars, 30.29%, 26.43%, 13.85%, 5.57%, 2.99%, 1.74%, 1.23%, 1.02%, 0.78%, 16.10%, Items in Top 300, 87, 50, 40, 23, 16, 11, 7, 7, 5, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, Share of Units in Top 300, 34.92%, 34.41%, 15.22%, 5.14%, 1.56%, 1.16%, 0.94%, 0.73%, 0.53%, 0.51%, 0.41%, 0.39%, 0.36%, 0.34%, 0.28%, 0.27%, 0.26%, 0.25%, 0.21%, 0.21%, 0.20%, 0.19%, 0.17%, 0.13%, 0.13%, 0.13%, ...)]
Index: []

### Processing

In [135]:
def _upload_data(bucket_name: str, blob_name: str, data: str):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_string(data, 'text/csv')

In [126]:
raw_monthly_sales_list = all_raw_monthly_sales()#path_filter = 'web_scrape/monthlycomicssales/2019')

/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/au

In [127]:
len(raw_monthly_sales_list)

265

In [137]:
# table 1 - single issues
single_issue_frames = []
for month in raw_monthly_sales_list:
#     print(month[1].columns)
    this_frame = month['data'][1].copy()
    
    this_frame['month'] = month['month']
    
    if 'Unit' in this_frame.columns:
        this_frame.rename(columns={"Unit": "Units"}, inplace=True)
    if 'Dollar' in this_frame.columns:
        this_frame.rename(columns={"Dollar": "Dollars"}, inplace=True)
    if 'Unit sales' in this_frame.columns:
        this_frame.rename(columns={"Unit sales": "Units"}, inplace=True)
    if 'Dollar sales' in this_frame.columns:
        this_frame.rename(columns={"Dollar sales": "Dollars"}, inplace=True)
    
    single_issue_frames.append(this_frame)
    
single_issues = pd.concat(single_issue_frames, ignore_index=True, sort=False)

In [147]:
single_issues.head()

,Units,Dollars,Comic-book Title,Issue,Price,Publisher,Est. units,month,On sale,Fused
0,1,-,Uncanny X-Men,353,$1.99,Marvel,154418,1998-01,NaN,NaN
1,2,-,X-Men,73,$1.99,Marvel,148639,1998-01,NaN,NaN
2,3,-,Avengers,2,$1.99,Marvel,138884,1998-01,NaN,NaN
3,4,-,Spawn,70,$1.95,Image,138051,1998-01,NaN,NaN
4,5,-,Iron Man,2,$1.99,Marvel,129906,1998-01,NaN,NaN


In [164]:
single_issues.to_pickle('single_issues.pkl')

In [141]:
# table 3 - collections
collection_frames = []
for month in raw_monthly_sales_list:
    this_frame = month['data'][3].copy()
    
    this_frame['month'] = month['month']
    
#     if 'Unit' in this_frame.columns:
#         this_frame.rename(columns={"Unit": "Units"}, inplace=True)
#     if 'Dollar' in this_frame.columns:
#         this_frame.rename(columns={"Dollar": "Dollars"}, inplace=True)
#     if 'Unit sales' in this_frame.columns:
#         this_frame.rename(columns={"Unit sales": "Units"}, inplace=True)
#     if 'Dollar sales' in this_frame.columns:
#         this_frame.rename(columns={"Dollar sales": "Dollars"}, inplace=True)
    
    collection_frames.append(this_frame)
    
collections = pd.concat(collection_frames, ignore_index=True, sort=False)

In [165]:
collections.head()

,Units,Dollars,Trade Paperback title,Price,Publisher,Est. units,month
0,1,3,Divine Right Collected Edition #1,$5.95,Image,NaN,1998-01
1,2,4,Swamp Thing Roots,$7.95,DC,NaN,1998-01
2,3,8,Overstreet Comic Book Price Update #1,$2.95,Gemstone,NaN,1998-01
3,4,24,Lady Death Dan Edition #1,$7.50,Chaos,NaN,1998-01
4,5,5,What's Michael Michael's Mambo,$5.95,Dark Horse,NaN,1998-01


In [166]:
collections.to_pickle('collections.pkl')